In [1]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
import squidpy as sq
from skimage import io
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import anndata as ad
import cv2

In [2]:
p_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = p_dir / "09_datasets"

In [3]:
dataset_1 = data_dir / '01_codex_data'
dataset_2 = data_dir / '02_tissue_architecture'
dataset_3 = data_dir / '03_Sci_Data_Cycif' 
dataset_5 = data_dir / '05_MCMICRO' / 'TNP_pilot_cycif' 

mask_folder = data_dir / 'exported' / 'foll_masks'

In [7]:
masks_1_folder = r'Y:\coskun-lab\Thomas\09_datasets\exported\masks\03_cycif'
masks = []
coords = []
sizes = []

for (dirpath, dirnames, filenames) in os.walk(masks_1_folder):
    for name in tqdm(sorted(filenames)):
        if 'tif' not in name:
            continue
        
        # read mask
        path = os.path.join(dirpath, name)
        mask = io.imread(path)
        
        # If non cell in mask pass
        if mask.max() == 0:
            continue
        
        # Get mask coords and size
        info = name.split(' ')[1]
        info = info.split('_')
        row = int(info[1][2:])
        col = int(info[0][2:])
        size = (int(info[3][2:]), int(info[2][2:]))
                
        masks.append(mask)
        coords.append((row, col))
        sizes.append(size)

coords = np.array(coords)
sizes = np.array(sizes)

  0%|          | 0/55 [00:00<?, ?it/s]

In [8]:
max_coords = np.max(coords+sizes, axis=0)

mask_sti = np.zeros((max_coords[0], max_coords[1]), np.uint32)
max_counter = 0
for i, coord in tqdm(enumerate(coords)):
    mask = np.where(masks[i]>0, masks[i] + max_counter, 0)
    mask_sti[coord[0]:coord[0]+sizes[i][0], coord[1]:coord[1]+sizes[i][1]] = masks[i]
    max_counter += np.max(masks[i])

0it [00:00, ?it/s]

In [10]:
import napari
napari.view_image(mask_sti)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 4799.5, 7199.5), zoom=0.092328125, angles=(0.0, 0.0, 90.0), perspective=0, interactive=True), cursor=Cursor(position=(1, 1), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 9600.0, 1.0), (0.0, 14400.0, 1.0)), current_step=(4800, 7200), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'mask_sti' at 0x1d238485a80>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_box_drag=None, selectio